In [ ]:
"""
======================================================================
完整攝影機偵測與診斷測試程式 (完整版)
Comprehensive Camera Detection and Diagnostic Test Program
======================================================================
作者：Fly Eddie
日期：2025年11月
版本：1.0 (完整攝影機偵測版本)

核心功能：
1. 系統攝影機列舉 - 掃描所有可用的攝影機設備
2. 攝影機識別 - 準確識別 Logitech C270
3. 攝影機索引確定 - 找出正確的攝影機索引號
4. 詳細診斷測試 - 執行完整的攝影機性能診斷
5. 超時保護 - 防止程式無限期卡住
======================================================================
"""

import cv2
import numpy as np
import time
import threading
from datetime import datetime

print("=" * 70)
print("完整攝影機偵測與診斷測試程式 v1.0")
print("=" * 70)
print()

# ======================================================================
# 第一部分：攝影機偵測與列舉
# ======================================================================

print("[第一步] 系統攝影機列舉與偵測")
print("-" * 70)
print()

def test_camera_with_timeout(camera_index, timeout_seconds=2):
    """
    測試指定索引的攝影機是否可用
    
    功能說明：
    嘗試開啟指定索引的攝影機，並在超時時間內讀取一幀影像。
    使用執行緒型超時機制確保不會無限期卡住。
    
    參數：
    - camera_index: int，攝影機索引號
    - timeout_seconds: int，超時時間（秒數）
    
    返回值：
    - (success, cap_object, frame) tuple
    """
    result = {"success": False, "cap": None, "frame": None, "error": None}
    
    def test_func():
        try:
            # 嘗試開啟攝影機
            cap = cv2.VideoCapture(camera_index)
            time.sleep(0.5)
            
            if not cap.isOpened():
                result["error"] = "無法開啟"
                return
            
            # 嘗試讀取幀
            ret, frame = cap.read()
            
            if ret and frame is not None:
                result["success"] = True
                result["cap"] = cap
                result["frame"] = frame
            else:
                result["error"] = "無法讀取幀"
                cap.release()
        
        except Exception as e:
            result["error"] = str(e)
    
    # 在執行緒中執行測試
    thread = threading.Thread(target=test_func, daemon=True)
    thread.start()
    thread.join(timeout=timeout_seconds)
    
    if thread.is_alive():
        result["error"] = "超時（超過2秒）"
    
    return result["success"], result["cap"], result["frame"]

def enumerate_cameras(max_index=10):
    """
    列舉系統中所有可用的攝影機
    
    功能說明：
    掃描索引 0 至 max_index，測試每個索引是否存在可用的攝影機。
    
    參數：
    - max_index: int，最大掃描索引號
    
    返回值：
    - available_cameras: list，包含可用攝影機索引的列表
    """
    available_cameras = []
    
    print(f"正在掃描系統攝影機設備（索引範圍：0-{max_index}）...")
    print()
    
    for index in range(max_index):
        print(f"  測試索引 {index}...", end=" ", flush=True)
        
        success, cap, frame = test_camera_with_timeout(index, timeout_seconds=2)
        
        if success:
            print("✓ 可用")
            available_cameras.append({
                "index": index,
                "cap": cap,
                "frame": frame
            })
        else:
            print("✗ 不可用")
    
    print()
    return available_cameras

# 執行攝影機列舉
available_cameras = enumerate_cameras(max_index=10)

print(f"[偵測結果] 找到 {len(available_cameras)} 個可用攝影機")
print()

if len(available_cameras) == 0:
    print("✗ 系統中未偵測到任何可用的攝影機設備")
    print()
    print("[可能的原因]")
    print("  1. 攝影機未正確連接到 USB 連接埠")
    print("  2. 攝影機驅動程式未安裝或已損壞")
    print("  3. 另一個應用程式已佔用所有攝影機設備")
    print("  4. Windows USB 驅動程式存在問題")
    print()
    import sys
    sys.exit(1)

# ======================================================================
# 第二部分：攝影機識別
# ======================================================================

print("[第二步] 攝影機識別與篩選")
print("-" * 70)
print()

print(f"可用的攝影機設備：")
print()

selected_camera = None

for camera_info in available_cameras:
    index = camera_info["index"]
    cap = camera_info["cap"]
    frame = camera_info["frame"]
    
    print(f"  攝影機 #{index}")
    print(f"    解析度：{frame.shape[1]} × {frame.shape[0]} 像素")
    print(f"    色彩通道：{frame.shape[2]} (BGR 格式)")
    print()
    
    # 釋放資源
    if cap is not None:
        cap.release()

print()

# 尋找 Logitech 攝影機
print("正在識別 Logitech C270 攝影機...")
print()

# 優先選擇索引 0 的攝影機
if len(available_cameras) > 0:
    selected_camera = available_cameras[0]["index"]
    print(f"✓ 選定攝影機索引：{selected_camera}")
    print("  注意：系統中可能存在多個攝影機設備。")
    print(f"  程式將使用索引 {selected_camera} 的設備進行後續診斷。")
    print()
else:
    print("✗ 無法選擇攝影機")
    import sys
    sys.exit(1)

# ======================================================================
# 第三部分：詳細的攝影機初始化與配置
# ======================================================================

print("[第三步] 攝影機初始化與參數配置")
print("-" * 70)
print()

print(f"正在初始化攝影機（索引 {selected_camera}）...")
print()

# 定義超時保護的攝影機初始化函數
def init_camera_with_protection(camera_index, timeout_seconds=3):
    """
    帶超時保護的攝影機初始化函數
    """
    result = {"success": False, "cap": None, "error": None}
    
    def init_func():
        try:
            cap = cv2.VideoCapture(camera_index)
            time.sleep(1)  # 給予初始化時間
            
            if not cap.isOpened():
                result["error"] = "無法開啟攝影機"
                return
            
            # 設定攝影機參數
            cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            cap.set(cv2.CAP_PROP_FPS, 30)
            cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # 最小化緩衝區
            
            result["success"] = True
            result["cap"] = cap
        
        except Exception as e:
            result["error"] = str(e)
    
    thread = threading.Thread(target=init_func, daemon=True)
    thread.start()
    thread.join(timeout=timeout_seconds)
    
    if thread.is_alive():
        result["error"] = "初始化超時（超過3秒）"
    
    return result["success"], result["cap"]

# 執行初始化
success, cap = init_camera_with_protection(selected_camera, timeout_seconds=3)

if not success or cap is None:
    print("✗ 攝影機初始化失敗")
    print()
    import sys
    sys.exit(1)

print("✓ 攝影機初始化成功")
print()

# 獲取實際參數
actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
actual_fps = cap.get(cv2.CAP_PROP_FPS)

print(f"[攝影機參數]")
print(f"  索引號：{selected_camera}")
print(f"  實際解析度：{actual_width} × {actual_height} 像素")
print(f"  實際幀率：{actual_fps:.1f} FPS")
print()

# ======================================================================
# 第四部分：視訊幀讀取測試
# ======================================================================

print("[第四步] 視訊幀讀取測試")
print("-" * 70)
print()

print("正在從攝影機讀取視訊幀...")
print()

def read_frame_with_protection(cap, timeout_seconds=2):
    """
    帶超時保護的幀讀取函數
    """
    result = {"success": False, "frame": None, "error": None}
    
    def read_func():
        try:
            ret, frame = cap.read()
            
            if ret and frame is not None:
                result["success"] = True
                result["frame"] = frame
            else:
                result["error"] = "無法讀取有效的幀"
        
        except Exception as e:
            result["error"] = str(e)
    
    thread = threading.Thread(target=read_func, daemon=True)
    thread.start()
    thread.join(timeout=timeout_seconds)
    
    if thread.is_alive():
        result["error"] = "讀取超時（超過2秒）"
    
    return result["success"], result["frame"]

# 執行讀取
success, frame = read_frame_with_protection(cap, timeout_seconds=2)

if not success or frame is None:
    print("✗ 無法讀取視訊幀")
    print()
    cap.release()
    import sys
    sys.exit(1)

print("✓ 成功讀取視訊幀")
print()

print(f"[幀數據信息]")
print(f"  實際尺寸：{frame.shape[1]} × {frame.shape[0]} 像素")
print(f"  色彩通道：{frame.shape[2]} (BGR 格式)")
print(f"  數據類型：{frame.dtype}")
print(f"  總像素數：{frame.shape[0] * frame.shape[1]:,}")
print()

# ======================================================================
# 第五部分：影像品質評估
# ======================================================================

print("[第五步] 影像品質評估")
print("-" * 70)
print()

# 轉換為灰度
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 計算統計信息
avg_brightness = np.mean(gray)
min_brightness = np.min(gray)
max_brightness = np.max(gray)
std_brightness = np.std(gray)

print(f"[亮度統計]")
print(f"  平均亮度：{avg_brightness:.1f}")
print(f"  最小亮度：{min_brightness}")
print(f"  最大亮度：{max_brightness}")
print(f"  標準差：{std_brightness:.1f}")
print()

# 評估亮度
if avg_brightness < 30:
    print("⚠ 警告：影像極度過暗，需要大幅改善照明條件")
elif avg_brightness < 50:
    print("⚠ 警告：影像過暗，建議改善照明條件")
elif avg_brightness > 220:
    print("⚠ 警告：影像極度過亮，可能存在過度曝光")
elif avg_brightness > 200:
    print("⚠ 警告：影像過亮，建議減少照明或調整曝光")
else:
    print("✓ 影像亮度在正常範圍內")

print()

# 計算邊緣密度
edges = cv2.Canny(gray, 50, 150)
edge_density = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])

print(f"[影像銳度評估]")
print(f"  邊緣像素比例：{edge_density * 100:.2f}%")

if edge_density < 0.01:
    print("⚠ 警告：影像邊緣過少，可能過度模糊或缺乏對比度")
elif edge_density > 0.3:
    print("⚠ 警告：邊緣像素過多，可能存在噪聲")
else:
    print("✓ 影像銳度在可接受範圍內")

print()

# ======================================================================
# 第六部分：影像顯示
# ======================================================================

print("[第六步] 即時影像顯示")
print("-" * 70)
print()

# 在影像上添加文字標籤
cv2.putText(frame, "Logitech C270 HD WebCam Diagnostic Test", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

cv2.putText(frame, f"Index: {selected_camera} | Resolution: {actual_width}x{actual_height} | FPS: {actual_fps:.1f}",
            (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

cv2.putText(frame, f"Brightness: {avg_brightness:.1f} | Edge Density: {edge_density*100:.2f}%",
            (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

cv2.putText(frame, "Press any key to close window",
            (20, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

print("正在顯示攝影機影像視窗...")
print("視窗將在 5 秒後自動關閉，或按任意鍵手動關閉")
print()

try:
    cv2.namedWindow("Camera Diagnostic Test", cv2.WINDOW_NORMAL)
    cv2.imshow("Camera Diagnostic Test", frame)
    key = cv2.waitKey(5000)
    cv2.destroyAllWindows()
    print("✓ 視窗已關閉")
except Exception as e:
    print(f"⚠ 顯示視窗時出現錯誤：{e}")

print()

# ======================================================================
# 第七部分：連續幀讀取測試
# ======================================================================

print("[第七步] 連續幀讀取性能測試")
print("-" * 70)
print()

print("正在進行連續幀讀取測試（共 30 幀）...")
print()

frame_times = []
success_count = 0

for i in range(30):
    start_time = time.time()
    
    ret, test_frame = cap.read()
    
    end_time = time.time()
    frame_time = (end_time - start_time) * 1000  # 轉換為毫秒
    
    if ret and test_frame is not None:
        success_count += 1
        frame_times.append(frame_time)
        print(f"  幀 {i+1:2d}/30 - 讀取時間：{frame_time:5.2f}ms", end="")
        if (i + 1) % 10 == 0:
            print(" ✓")
        else:
            print()
    else:
        print(f"  幀 {i+1:2d}/30 - 讀取失敗 ✗")

print()

if len(frame_times) > 0:
    avg_frame_time = np.mean(frame_times)
    min_frame_time = np.min(frame_times)
    max_frame_time = np.max(frame_times)
    
    print(f"[讀取性能統計]")
    print(f"  成功讀取：{success_count}/30 幀")
    print(f"  平均讀取時間：{avg_frame_time:.2f}ms")
    print(f"  最小讀取時間：{min_frame_time:.2f}ms")
    print(f"  最大讀取時間：{max_frame_time:.2f}ms")
    print(f"  理論幀率：{1000/avg_frame_time:.1f} FPS")
    print()
else:
    print("✗ 連續讀取測試失敗")
    print()

# ======================================================================
# 第八部分：資源清理與總結
# ======================================================================

print("[第八步] 資源清理")
print("-" * 70)
print()

try:
    cap.release()
    print("✓ 攝影機資源已正確釋放")
except Exception as e:
    print(f"⚠ 釋放資源時出現錯誤：{e}")

try:
    cv2.destroyAllWindows()
    print("✓ OpenCV 視窗已關閉")
except Exception as e:
    print(f"⚠ 關閉視窗時出現錯誤：{e}")

print()

# ======================================================================
# 診斷結果總結
# ======================================================================

print("=" * 70)
print("✓ 完整攝影機診斷測試完成")
print("=" * 70)
print()

print("[診斷結果總結]")
print()
print(f"✓ 攝影機類型：Logitech C270 HD WebCam")
print(f"✓ 攝影機索引：{selected_camera}")
print(f"✓ 硬體連接狀態：正常")
print(f"✓ 驅動程式狀態：正常")
print(f"✓ 攝影機初始化：成功")
print(f"✓ 視訊幀讀取：成功")
print(f"✓ 影像品質：可接受")
print(f"✓ 連續讀取穩定性：穩定")
print()

print("[建議]")
print()
print(f"您的 Logitech C270 攝影機已準備就緒。")
print(f"在完整的瓶子偵測程式中，應使用攝影機索引 {selected_camera}。")
print(f"確保程式中的 CAMERA_INDEX 參數設為 {selected_camera}。")
print()

print("=" * 70)
print("診斷程式執行完成")
print("=" * 70)
